# ブロックとProc

* ブロック
    * 新たにスコープを作成する
        * for式、if式、while式などはスコープは作成されない
    * <font color="red">メソッドを呼び出すときのみ記述できる</font>
    * yieldという式を使う事でブロック内部で記述した式を呼び出すことができる
    * 「{}」と「do ～ end」の形式で書ける
        * 傾向
            * 1行で書けるときは{}
            * 複数行で書く時は do end
            
* 参考
    * [メタプログラミングRuby 第3章 まとめ　(ブロック)](http://portaltan.hatenablog.com/entry/2015/07/17/181358)

## yieldで実行結果の取得

* 「{}」に囲まれているのがブロック
    * ここのブロックは2を返す
    * メソッドfunc
        * ブロックの実行結果(yield)と引数の合計3を返している

In [13]:
def func x
  x + yield
end
puts func(1){ 2 }

### do ～ end の形式でも書ける
# func(1) do
#   2
# end

3


## スコープの生成

* ブロックの中で変数に代入された値はブロックの外では参照できない

In [27]:
def func x
  x + yield
end
func(1){ z = 2 }

### ブロック内で作成された変数zはブロックの外では参照できない
puts z

NameError: undefined local variable or method `z' for main:Object

## クロージャとしてのブロック

* クロージャ
    * 処理の生成時の環境を束縛するもの
* おそらく
    * クロージャはブロック(メソッド)の外で作った変数を利用しているブロックのこと
    * 束縛はクロージャで使う変数をそれっぽく難しく言ったもの
* 挙動
    * ブロックの外で変数yに2を代入 (束縛yと表現する？)
    * メソッドfuncにブロックを渡し、yieldによりブロックを実行
        * このときyの値を取得して更新する
        * このyは、ブロックの外と同じy
            * <font color="red">値ではなく変数そのものが共有されている</font>
* Rubyはメソッド内部から外部の変数を参照できない
    * クロージャが呼び出し元の変数を処理に持ち込む数少ない手段の一つ

In [24]:
def func x
  x + yield
end

### ブロックの外で変数yに値を代入
y = 2

### このブロック内のyは先ほどのブロックの外の変数yと同一
puts func(1){y+=2}

### yの値は更新されている
puts y

5
4


## ブロックの引数の指定

* funcに1と2を渡す
* func内部で以下合計する
    * 第一引数の値1
    * ブロックの実行結果
        * yieldはブロック引数2,3を合計して5を返す
* 6が返る

In [28]:
def func a, b
  a + yield(b, 3)
end

puts func(1, 2){|x,y| x+y}

6


## ブロックの判定(ブロックが指定されたかどうか)

* block_given?メソッドで実現できる

In [29]:
def func
  return 1 if block_given?
  2
end

puts func(){}
puts func

1
2


## Procの基本

* ブロックをオブジェクトとして渡したい時に利用する
    * ブロックそのものはオブジェクトではない
* 生成
    * Procクラスのコンクラスタにブロックを指定すると生成できる
* 利用
    * Procのインスタンスに対してcallメソッドを使う


In [30]:
proc = Proc.new{|x| puts x}
proc.call(1)

1


## Procオブジェクトの生成

* 遅延評価
    * 処理自体を先に生成して後で評価する
    * 例
        * カウンタで初期値をプログラム冒頭で決定 : 処理と初期値の設定
        * 後の処理でカウントする : 遅延評価

In [44]:
def get_counter start
  Proc.new do |up|
    puts "start= #{start}"
    puts "up= #{up}"
    start += up
  end
end

### 初期値として3を設定。count_upはProcオブジェクトを参照
count_up = get_counter(3)

### 初期値に1を足す
puts count_up.call(1)

### さらに3を足す
puts count_up.call(3)

start= 3
up= 1
4
start= 4
up= 3
7


## Proc ⇔ ブロック 変換

* Proc -> ブロック
    * Procオブジェクトに「&」をつけて最後の引数に指定する
* ブロック -> Proc
    * 最後の仮引数に「&」をつけた引数で参照できる

In [55]:
### Proc -> ブロック
def func x
  x + yield
end
proc = Proc.new{2}

### 最後の引数に&をつけて渡す
puts func(1,&proc)
  
#=========================#
  
### ブロック -> Proc
### 関数の最後の仮引数に&をつけた名前を渡す
def func2 y, &proc2
  y + proc2.call
end
puts func2(3){ 4 }

3
7


## lambda


* lambdaメソッド
    * Procインスタンスを生成する
        * Procインスタンスはlambdaとも呼ばれる
        * Procやブロックよりもメソッドに近い動きをする
    * Proc.newで生成したインスタンはprocとも呼ばれる

In [43]:
lmd = lambda{|x| puts x}
lmd.call(1)

### Ruby1.9以降の新しい書き方 (アロー演算子)
### putsを外出し
lmd2 = -> ( x ) { x }
puts lmd2.call(2)

### putsをブロック内に記載
lmd3 = -> (x, y){puts x + y}
lmd3.call(3, 4)

1
2
7


## Procとlambdaの振る舞いの違い

* returnとスコープ
    * Procは生成元のスコープを脱出する
    * lambdaはブロック内でreturnすると呼び出し元に復帰する

In [25]:
puts "Procはreturnでスコープを脱出するので2まで行かない"
def func_proc
  proc = Proc.new{return 1}
  puts proc.call
  puts 2
 end
puts func_proc

puts "lambdaは呼び出し元に戻るので4まで行く"
def func_lambda
  proc = lambda{return 3}
  puts proc.call
  puts 4
end
func_lambda

Procはcallでスコープを脱出するので2まで行かない
1
lambdaは呼び出し元に戻る
3
4


* 引数が一致しない場合
    * Procは無視、もしくはnilを返す
    * lambdaは例外ArgumentErrorが発生する

In [39]:
puts "引数が一致しない場合はnil"
p1 = Proc.new{|x,y| y}
p p1.call(1,2)
p p1.call(1)

puts "引数が一致しない場合は例外が発生"
l1 = lambda{|x,y| y}
p l1.call(3,4)
p l1.call(3)

引数が一致しない場合はnil
2
nil
引数が一致しない場合は例外が発生
4


ArgumentError: wrong number of arguments (given 1, expected 2)

# 配列やハッシュの走査

* for式、while式はスコープが作成されないので走査には余り使われない
    * 代わりにブロックを受け取るメソッドが使われる
* ブロックを受け取るメソッド
    * each
    * each_with_index
    * each_key
    * each_value
    * upto
    * dwonto
    * times



## 配列のeach

* each
    * メソッドのブロックの引数には要素の値が格納される
* each_with_index
    * 引数を２つ取る
        * 第一引数は要素の値
        * 第二引数はインデックス
            * 0からはじまる

In [50]:
[1,2,3].each do |value|
  puts "value = #{value}"
end

[4,5,6].each_with_index do |value, index|
  puts "index = #{index}, value = #{value}"
end

value = 1
value = 2
value = 3
index = 0, value = 4
index = 1, value = 5
index = 2, value = 6


[4, 5, 6]

## ハッシュのeach

* each
    * 引数を２つ取る
        * 第一引数はキー
        * 第二引数はバリュー
* each_key
    * キーのみ
* each_value
    * バリューのみ

In [61]:
{a: 1, b: 2}.each do |key, value|
  puts "key = #{key}, value = #{value}"
end

puts "引数が一つの場合はキーとバリューの配列ができる"
{a: 1, b: 2}.each do |x|
  puts "x = #{x}"
  p x[0]
end

{c: 3, d: 4}.each_key do |key|
  puts "key = #{key}"
end

{e: 5, f: 6}.each_value do |value|
  puts "value = #{value}"
end

key = a, value = 1
key = b, value = 2
引数が一つの場合はキーとバリューの配列ができる
x = [:a, 1]
:a
x = [:b, 2]
:b
key = c
key = d
value = 5
value = 6


{:e=>5, :f=>6}

## 範囲オブジェクトのeach

範囲内を繰り返す

In [65]:
("a".."d").each do |value|
  puts value
end

("1"..."4").each do |value|
  puts value
end

a
b
c
d
1
2
3


"1"..."4"

## upto, downto,timesメソッド

In [71]:
2.upto(4) do |x|
  puts x
end

8.downto(6) do |x|
  puts x
end

3.times do |x|
  puts "Hello"
end

2
3
4
8
7
6
Hello
Hello
Hello


3

# スレッド

* Threadクラスを使ってマルチスレッドプログラムを書くことができる
    * インスタンス化することで新しいスレッドを生成できる
    * joinメソッドでスレッドの終了を待つことができる

In [81]:
# t = Thread.new do
#   p "start"
#   sleep 5
#   p "end"
# end
# p "wait"
# t.join

3.times do |i|
  Thread.start(i) do |index|
    puts "thread-#{index} start"
  end
end

### pryだと以下のように出力される
# ===
# thread-0 start
# thread-1 start
# => 3
# [5] pry(main)> thread-2 start
# ===

3

## ファイバ

* スレッドと同じように複数のタスクを切り替えて並列処理をするための機能
    * Ruby1.9から
* スレッドと異なる点
    * スレッドはタスクの切り替えをOSや仮想マシンが行う
    * ファイバは切替のタイミングを開発者がプログラム内で明示的に記述する
* 他言語ではコルーチンと呼ぶこともある
* resumeメソッド
    * ファイバへのコンテキストを切り替える
    * Fiber.yieldが呼び出されるまでファイバ内の処理を実行する

In [86]:
f = Fiber.new do
  (1..3).each do |i|
    Fiber.yield i
  end
  nil
end

p f.resume
p f.resume
p f.resume
p f.resume
p f.resume

1
2
3
nil


FiberError: dead fiber called